#### Construction of Index Benefit Graph (from Schnaitter's PhD Thesis, 2011)

***Definition***: The `IBG` of a query $q$ is a `DAG` in which each node $Y$ is a subset of $C$, a set of all relevant indexes that could ever be utlilized in the execution of $q$. Node $Y$ also stores the following two quantities: 

* $cost(q,Y)$ which is the query optimizer's estimated cost for executing $q$ under configuration $Y$  
* $used(q,Y)$ which is the subset of indexes from $Y$ that are included in the query plan


Recursive algorithm for constructing the IBG:

```python
construct_IBG(q, Y):
    if Y.built:
        return

    # obtain estimated cost and determine indexes used
    Y.cost = cost(q,Y)
    Y.used = used(q,Y)
    Y.built = True
    
    # create children (one for each index in Y.used)
    for a in Y.used:
        create child node: X = Y - {a}   # child node is set Y with index a removed
        X.built = False
        Y.add_child(X)
        # recursively construct IBG on children
        construct_IBG(q, X)

```




```python
# create root node
Y = C
Y.built = False

# call construct_IBG(q, Y)
construct_IBG(q, Y)
```


It is possible that some nodes may share the same child. Instead of creating a new copy of that child for each different parent node, we can use a separate hash table to keep track of children that have already been created and reuse those children.
